In [13]:
## Import Required library
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import fnmatch
import matplotlib.pyplot as plt

In [2]:
## Working Directory
os.chdir('F:\\Northeastern University\\sem2\\data mining\\project\\final')
cwd = os.getcwd()
print(cwd)

F:\Northeastern University\sem2\data mining\project\final


In [3]:
## Reading the csv file
df = pd.DataFrame()
df = pd.read_csv('DSI_kickstarterscrape_dataset1.csv')
print(df.head())

   project id                                               name  \
0       39409                              WHILE THE TREES SLEEP   
1      126581               Educational Online Trading Card Game   
2      138119                                              STRUM   
3      237090  GETTING OVER - One son's search to finally kno...   
4      246101  The Launch of FlyeGrlRoyalty &quot;The New Nam...   

                                                 url      category  \
0  http://www.kickstarter.com/projects/emiliesaba...  Film & Video   
1  http://www.kickstarter.com/projects/972789543/...         Games   
2  http://www.kickstarter.com/projects/185476022/...  Film & Video   
3  http://www.kickstarter.com/projects/charnick/g...  Film & Video   
4  http://www.kickstarter.com/projects/flyegrlroy...       Fashion   

          subcategory         location      status     goal  pledged  \
0          Short Film     Columbia, MO  successful  10500.0  11545.0   
1  Board & Card Games    M

In [4]:
## Dimension
print(df.shape)
print(df.dtypes)

(45957, 17)
project id             int64
name                  object
url                   object
category              object
subcategory           object
location              object
status                object
goal                 float64
pledged              float64
funded percentage    float64
backers                int64
funded date           object
levels                 int64
reward levels         object
updates                int64
comments               int64
duration             float64
dtype: object


In [5]:
## Removing unwanted rows:
print(df.status.value_counts())

df.drop(df.loc[df['status']=='live'].index, inplace=True)
df.drop(df.loc[df['status']=='canceled'].index, inplace=True)
df.drop(df.loc[df['status']=='suspended'].index, inplace=True)

successful    22969
failed        18996
live           3929
canceled         59
suspended         4
Name: status, dtype: int64


In [6]:
## Spliting the column into two:
new = df["location"].str.split(", ", n = 1, expand = True)
df["city"]= new[0]
df["state"]= new[1]
df.drop(columns =["location"], inplace = True)

print(df.shape)

(41965, 18)


In [7]:
##Filtering states
for each in df.state:
    if (fnmatch.fnmatch(str(each),'??')) == False:
        df.drop(df.loc[df['state']== each].index, inplace=True)

print(df.shape)

(39855, 18)


In [8]:
## Checking for missing values:
print(df.isnull().sum())
df = df.dropna()
print(df.isnull().sum())

print(df.shape)

project id              0
name                    0
url                     0
category                0
subcategory             0
status                  0
goal                    0
pledged                 0
funded percentage       0
backers                 0
funded date             0
levels                  0
reward levels          57
updates                 0
comments                0
duration                0
city                 1322
state                1323
dtype: int64
project id           0
name                 0
url                  0
category             0
subcategory          0
status               0
goal                 0
pledged              0
funded percentage    0
backers              0
funded date          0
levels               0
reward levels        0
updates              0
comments             0
duration             0
city                 0
state                0
dtype: int64
(38491, 18)


In [9]:
## Finding Main Category, Sub Category and number of states:
print(len(df.category.unique()), "Main categories\n")
print(df.category.value_counts())
print(len(df.subcategory.unique()), "sub categories\n")
print(df.subcategory.value_counts())
print(len(df.state.unique()), "Number of states\n")
print(df.state.value_counts())

14 Main categories

Film &amp; Video    10925
Music                9559
Publishing           3778
Art                  3263
Theater              2196
Design               1450
Games                1365
Food                 1243
Photography          1110
Fashion               975
Comics                913
Technology            654
Dance                 649
Film & Video          411
Name: category, dtype: int64
51 sub categories

Short Film                3517
Documentary               3060
Music                     2876
Theater                   2196
Film &amp; Video          2162
Indie Rock                1732
Rock                      1584
Narrative Film            1294
Food                      1243
Photography               1110
Fashion                    975
Webseries                  960
Comics                     913
Fiction                    885
Art                        872
Product Design             850
Nonfiction                 789
Country &amp; Folk         715
Video Game

### Implementing k - Nearest Neighbour Algorithm:

In [10]:
## Converting categorical data into numerical data:
df.category = pd.Categorical(df.category).codes
df.state = pd.Categorical(df.state).codes
df.city = pd.Categorical(df.city).codes
df.subcategory = pd.Categorical(df.subcategory).codes
df.status = pd.Categorical(df.status).codes

In [11]:
## Dividing into training and test data set
y1 = df['status']
df2 = df[['category','subcategory','city','state','goal','levels','duration','updates']]
x_trn, x_tst, y_trn, y_tst = train_test_split(df2,y1, test_size = 0.25)

In [15]:
## k - Nearest Neighbour Model:
knn = KNeighborsClassifier(n_neighbors=7)

In [16]:
## Train the model using training data set
knn.fit(x_trn, y_trn)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=7, p=2,
           weights='uniform')

In [17]:
## Prediction on test data set
y_pred = knn.predict(x_tst)

In [18]:
## Model Accuracy and F1-Score
cfm = metrics.confusion_matrix(y_tst,y_pred)
acc = metrics.accuracy_score(y_tst, y_pred)
f1_score = metrics.f1_score(y_tst,y_pred)
report = metrics.classification_report(y_tst,y_pred)
print("Confusion  Matrix", cfm)
print("Accuracy:",acc*100)
print("Error:", (1 - acc)*100)
print("F1 Score:",f1_score)
print("Classfication Report:",report)

Confusion  Matrix [[2349 1990]
 [1516 3768]]
Accuracy: 63.566455367349064
Error: 36.433544632650936
F1 Score: 0.6824850570548814
Classfication Report:               precision    recall  f1-score   support

           0       0.61      0.54      0.57      4339
           1       0.65      0.71      0.68      5284

   micro avg       0.64      0.64      0.64      9623
   macro avg       0.63      0.63      0.63      9623
weighted avg       0.63      0.64      0.63      9623



In [19]:
### Model with selected features

## Dividing the data set into training and test data
y2 = df['status']
df3 = df[['category','subcategory','city','state','goal','levels','duration','updates']]
x_trn, x_tst, y_trn, y_tst = train_test_split(df3,y2, test_size = 0.25)

In [20]:
## kNN Model:
knn = KNeighborsClassifier(n_neighbors=7)

In [21]:
## Train the model using training data set
knn.fit(x_trn, y_trn)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=7, p=2,
           weights='uniform')

In [22]:
## Prediction on test data set
y_pred = knn.predict(x_tst)

In [23]:
## Model Accuracy and F1-Score
cfm = metrics.confusion_matrix(y_tst,y_pred)
acc = metrics.accuracy_score(y_tst, y_pred)
f1_score = metrics.f1_score(y_tst,y_pred)
report = metrics.classification_report(y_tst,y_pred)
print("Confusion  Matrix", cfm)
print("Accuracy:",acc*100)
print("Error:", (1 - acc)*100)
print("F1 Score:",f1_score)
print("Classfication Report:",report)

Confusion  Matrix [[2332 2004]
 [1566 3721]]
Accuracy: 62.90138210537255
Error: 37.09861789462745
F1 Score: 0.6758082092262987
Classfication Report:               precision    recall  f1-score   support

           0       0.60      0.54      0.57      4336
           1       0.65      0.70      0.68      5287

   micro avg       0.63      0.63      0.63      9623
   macro avg       0.62      0.62      0.62      9623
weighted avg       0.63      0.63      0.63      9623

